In [1]:

from tensorflow import keras
from keras.layers import Embedding,LSTM,Dropout,Dense,Flatten
import pandas as pd
from keras.models import Sequential
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [3]:
data=pd.read_csv("C:/Users/AKASH/Desktop/twitter_training.csv")


In [4]:
x=data["im getting on borderlands and i will murder you all ,"]
y=data['Positive']

x

0        I am coming to the borders and I will kill you...
1        im getting on borderlands and i will kill you ...
2        im coming on borderlands and i will murder you...
3        im getting on borderlands 2 and i will murder ...
4        im getting into borderlands and i can murder y...
                               ...                        
74676    Just realized that the Windows partition of my...
74677    Just realized that my Mac window partition is ...
74678    Just realized the windows partition of my Mac ...
74679    Just realized between the windows partition of...
74680    Just like the windows partition of my Mac is l...
Name: im getting on borderlands and i will murder you all ,, Length: 74681, dtype: object

In [5]:
y=np.array([0 if label=="Positive" else
            1 if label=="neutral" else
            2 if label=="Negative" else
            3 if label=='Irrelevant' else
            3 for label in y])



In [6]:
def preprocess_text(text):
    text=re.sub('[^a-zA-Z]',' ',text)
    text=text.lower().strip()
    
    return text
    

In [8]:
# Modify preprocessing loop to handle non-string values
x = [preprocess_text(str(sentence)) if isinstance(sentence, str) else preprocess_text('') for sentence in x]



In [9]:
max_token=200
vocb_size=15000

tokenizer =keras.preprocessing.text.Tokenizer(vocb_size)
tokenizer.fit_on_texts(x)

x_sequences=tokenizer.texts_to_sequences(x)
x_pad=keras.preprocessing.sequence.pad_sequences(x_sequences,max_token)


In [10]:
x_train,x_test,y_train,y_test = train_test_split(x_pad,y,test_size=0.3,random_state=35)

In [11]:
y_train

array([0, 0, 3, ..., 3, 3, 3])

In [12]:
embedding= 50

model=Sequential()

model.add(Embedding(input_dim=vocb_size,output_dim=embedding,input_length=max_token))

model.add(LSTM(units=16,return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=8,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=4))
model.add(Dropout(0.1))


model.add(Dropout(0.1))



model.add(Dense(4,activation="softmax"))

In [265]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 200, 50)           750000    
                                                                 
 lstm_59 (LSTM)              (None, 200, 16)           4288      
                                                                 
 dropout_63 (Dropout)        (None, 200, 16)           0         
                                                                 
 lstm_60 (LSTM)              (None, 200, 8)            800       
                                                                 
 dropout_64 (Dropout)        (None, 200, 8)            0         
                                                                 
 lstm_61 (LSTM)              (None, 4)                 208       
                                                                 
 dropout_65 (Dropout)        (None, 4)               

In [15]:
opt =keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=['accuracy'])

In [16]:
model.fit(x_train,y_train,validation_data=(x_test,y_test), epochs=4,batch_size=100,shuffle=True,verbose=1)

Epoch 1/4
523/523 [==============================] - 265s 479ms/step - loss: 0.9979 - accuracy: 0.5083 - val_loss: 0.7722 - val_accuracy: 0.6881
Epoch 2/4
523/523 [==============================] - 246s 471ms/step - loss: 0.7122 - accuracy: 0.6908 - val_loss: 0.6024 - val_accuracy: 0.7748
Epoch 3/4
523/523 [==============================] - 216s 413ms/step - loss: 0.5851 - accuracy: 0.7437 - val_loss: 0.5330 - val_accuracy: 0.8049
Epoch 4/4
523/523 [==============================] - 259s 495ms/step - loss: 0.5086 - accuracy: 0.7690 - val_loss: 0.4736 - val_accuracy: 0.8224


In [24]:
sentence ="im getting on borderlands and i will kill you"
tokenizer.fit_on_texts(sentence)
x_sequences=tokenizer.texts_to_sequences([sentence])
x_pad=keras.preprocessing.sequence.pad_sequences(x_sequences,max_token)

result=model.predict(x_pad)
print(result)


1/1 [==============================] - 0s 60ms/step
[[0.86910385 0.00167295 0.03997139 0.08925176]]
